In [61]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta
import sqlite3

In [2]:
filters = {
    "Job Title": ["Product Manager", "Product Owner", "Technical Product Manager"],
    "Location": ["New York"],
    "Last Update": [(datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d %H:%M:%S")],
    "Keyword1": ["Agile", "Scrum", "Kanban"],
    "Keyword2": ["Roadmap", "Backlog", "Feature Prioritization"],
    "Keyword3": ["omnistudio", "apex", "Lightning Web Components"]
}

keywords = 3
matrix = pd.DataFrame(dict([(key, pd.Series(value)) for key, value in filters.items()]))
matrix = matrix.fillna("")

In [3]:
matrix.head()

,Job Title,Location,Last Update,Keyword1,Keyword2,Keyword3
0,Product Manager,New York,2025-01-06 16:35:48,Agile,Roadmap,omnistudio
1,Product Owner,,,Scrum,Backlog,apex
2,Technical Product Manager,,,Kanban,Feature Prioritization,Lightning Web Components


----

    Utility functions:

In [ ]:
import requests
import json
from datetime import datetime, timedelta

def run_api_search(filters, bearer):
    base_url = "https://api.coresignal.com/cdapi/v1/professional_network/employee/search/filter"

    # Ensure last_updated_gte is properly formatted
    last_update = filters.get("Last Update", "").strip("()")  # Remove surrounding parentheses
    if not last_update:
        last_update = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d %H:%M:%S")

    # Corrected payload
    payload = {
        "active_experience": True,
        "location": filters.get("Location", "").strip("()"),  # Remove parentheses
        "last_updated_gte": last_update,  # Ensure it's properly formatted
    }

    if "Job Title" in filters:
        payload["experience_title"] = filters["Job Title"]

    if "Keyword" in filters:
        payload["keyword"] = filters["Keyword"]

    headers = {
        'Content-Type': 'application/json',
        'Authorization': 'Bearer '
    }

    all_ids = []
    next_page_after = None

    while True:
        # If next_page_after is not None, ensure it's converted to a valid integer string
        if next_page_after is not None:
            try:
                next_page_after = str(int(next_page_after))  # Ensure it is a valid integer string
            except ValueError:
                print("Warning: Invalid pagination ID received. Stopping pagination.")
                break

            request_url = f"{base_url}?after={next_page_after}"
        else:
            request_url = base_url  # First request without pagination

        response = requests.post(request_url, headers=headers, data=json.dumps(payload))

        if response.status_code == 401:
            print("Error: Unauthorized - Check your API token.")
            return [], 0

        if response.status_code == 422:
            print(f"Error: API request failed with status {response.status_code} - Invalid pagination value.")
            break  # Stop pagination if API rejects the value

        if response.status_code != 200:
            print(f"Error: API request failed with status {response.status_code}")
            print(f"Response Text: {response.text}")
            break

        try:
            response_json = response.json()
            all_ids.extend(response_json)

            next_page_after = response.headers.get("x-next-page-after")

            # Ensure valid pagination value, otherwise stop looping
            if next_page_after is None or not next_page_after.isdigit():
                break  # Stop if no more pages or invalid value

        except json.JSONDecodeError:
            print("Error: Invalid JSON response")
            break

    return all_ids, len(all_ids)

In [ ]:
def collect_employees(ids):
    base_url = 'https://api.coresignal.com/cdapi/v1/professional_network/employee/collect'
    
    headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer '
    }

    profiles = {}

    for id in ids:
        url = f"{base_url}/{id}"
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            profiles[id] = response.json()
        else:
            print("Error", f"\nResponse:{response.text}")
    

    return profiles

In [46]:
def extract_linkedin_profile(profile):
    profile_data = list(profile.values())[0] if profile else {}

    extracted_data = {
        "id": profile_data.get("id", ""),
        "name": profile_data.get("name", ""),
        "title": profile_data.get("title", ""),
        "headline": profile_data.get("user_generated_headline", ""),
        "url": profile_data.get("url", ""),
        "shorthand_name": profile_data.get("member_shorthand_name", ""),
        "created": profile_data.get("created", ""),
        "last_updated": profile_data.get("last_updated", ""),
        "logo_url": profile_data.get("logo_url", ""),
        "summary": profile_data.get("summary", ""),
        "location": profile_data.get("location", ""),
        "country": profile_data.get("country", ""),
        "connections": profile_data.get("connections_count", 0),
        "recommendations": profile_data.get("recommendations_count", 0),
        "industry": profile_data.get("industry", ""),
        "experience_count": profile_data.get("experience_count", 0),
    }

    def extract_unique_entities(data_list, unique_keys):
        entities, seen = [], set()
        for item in data_list:
            if item.get("deleted", 0) == 1:
                continue
            identifier = tuple(item.get(k, "") for k in unique_keys)
            if identifier not in seen:
                seen.add(identifier)
                entities.append(item)
        return entities

    skills = []
    seen_skills = set()
    for skill_entry in profile_data.get("member_skills_collection", []):
        if skill_entry.get("deleted", 0) == 1:
            continue
        skill_data = skill_entry.get("member_skill_list", {})
        skill_id, skill_name = skill_data.get("id", ""), skill_data.get("skill", "")
        if skill_name and skill_id not in seen_skills:
            seen_skills.add(skill_id)
            skills.append({
                "id": skill_id,
                "skill_name": skill_name,
                "created": skill_data.get("created", ""),
                "last_updated": skill_data.get("last_updated", "")
            })

    extracted_data["experiences"] = extract_unique_entities(profile_data.get("member_experience_collection", []), ["title", "date_from"])
    extracted_data["education"] = extract_unique_entities(profile_data.get("member_education_collection", []), ["title", "subtitle"])
    extracted_data["awards"] = extract_unique_entities(profile_data.get("member_awards_collection", []), ["id"])
    extracted_data["certifications"] = extract_unique_entities(profile_data.get("member_certifications_collection", []), ["id"])
    extracted_data["courses"] = extract_unique_entities(profile_data.get("member_courses_collection", []), ["id"])
    extracted_data["groups"] = extract_unique_entities(profile_data.get("member_groups_collection", []), ["id"])
    extracted_data["interests"] = extract_unique_entities(profile_data.get("member_interests_collection", []), ["id"])
    extracted_data["languages"] = extract_unique_entities(profile_data.get("member_languages_collection", []), ["id"])
    extracted_data["organizations"] = extract_unique_entities(profile_data.get("member_organizations_collection", []), ["id"])
    extracted_data["patents"] = extract_unique_entities(profile_data.get("member_patents_collection", []), ["id"])
    extracted_data["projects"] = extract_unique_entities(profile_data.get("member_projects_collection", []), ["id"])
    extracted_data["publications"] = extract_unique_entities(profile_data.get("member_publications_collection", []), ["id"])
    extracted_data["recommendations"] = extract_unique_entities(profile_data.get("member_recommendations_collection", []), ["id"])
    extracted_data["skills"] = skills  # Fixed skills extraction
    extracted_data["test_scores"] = extract_unique_entities(profile_data.get("member_test_scores_collection", []), ["id"])
    extracted_data["volunteering_cares"] = extract_unique_entities(profile_data.get("member_volunteering_cares_collection", []), ["id"])
    extracted_data["volunteering_opportunities"] = extract_unique_entities(profile_data.get("member_volunteering_opportunities_collection", []), ["id"])
    extracted_data["volunteering_positions"] = extract_unique_entities(profile_data.get("member_volunteering_positions_collection", []), ["id"])
    extracted_data["volunteering_supports"] = extract_unique_entities(profile_data.get("member_volunteering_supports_collection", []), ["id"])

    return pd.DataFrame([extracted_data])

In [52]:
def store_profiles_in_db(df, db_path="profiles.db"):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Convert lists of dictionaries to JSON strings
    df = df.copy()
    for col in df.columns:
        if df[col].apply(lambda x: isinstance(x, list)).any():  # If any value in column is a list
            df[col] = df[col].apply(json.dumps)

    # Ensure table schema matches DF structure
    columns = ", ".join([f"{col} TEXT" for col in df.columns])
    cursor.execute(f"CREATE TABLE IF NOT EXISTS profiles ({columns})")

    for _, row in df.iterrows():
        cursor.execute("SELECT COUNT(*) FROM profiles WHERE url = ?", (row["url"],))
        if cursor.fetchone()[0] == 0:  # Profile does not exist, insert it
            placeholders = ", ".join(["?"] * len(df.columns))
            insert_query = f"INSERT INTO profiles ({', '.join(df.columns)}) VALUES ({placeholders})"
            cursor.execute(insert_query, tuple(row.fillna("")))  # Convert NaN to empty string

    conn.commit()
    conn.close()

    print("Profiles successfully stored in the database.")

---

    Tree creation:

In [12]:
def create_level_zero(matrix):
    # Extract static filters
    location = matrix["Location"].iloc[0]
    last_update = matrix["Last Update"].iloc[0]

    # Extract job titles and construct an OR filter
    job_titles = matrix["Job Title"].dropna().tolist()
    job_title_filter = " OR ".join([f'({title})' for title in job_titles]) if job_titles else ""

    if job_title_filter:
        job_title_filter = f"({job_title_filter})"

    root = {
        "filters": {
            "Location": f"({location})",
            "Last Update": f"({last_update})",
            "Job Title": job_title_filter
        },
        "children": []
    }

    return root

In [13]:
root = create_level_zero(matrix)
root

{'filters': {'Location': '(New York)',
  'Last Update': '(2025-01-04 23:38:20)',
  'Job Title': '((Product Manager) OR (Product Owner) OR (Technical Product Manager))'},
 'children': []}

In [14]:
def create_keyword_levels(root, matrix, keyword_index):
    keyword_col = f"Keyword{keyword_index}"
    keyword_values = matrix[keyword_col].dropna().tolist()
    keyword_filter = " OR ".join([f"({keyword})" for keyword in keyword_values]) if keyword_values else ""

    if keyword_filter:
        keyword_filter = f"({keyword_filter})"

    if not root["children"]:
        root["children"].append({
            "filters": {
                "Location": root["filters"]["Location"],
                "Last Update": root["filters"]["Last Update"],
                "Job Title": root["filters"]["Job Title"],
                "Keyword": keyword_filter
            },
            "children": []
        })
        return root

    latest_child = root["children"][-1]
    while latest_child["children"]:
        latest_child = latest_child["children"][-1]

    prev_keyword_filter = latest_child["filters"].get("Keyword", "")
    combined_keyword_filter = f"{prev_keyword_filter} AND {keyword_filter}" if prev_keyword_filter and keyword_filter else prev_keyword_filter or keyword_filter

    new_child = {
        "filters": {
            "Location": latest_child["filters"]["Location"],
            "Last Update": latest_child["filters"]["Last Update"],
            "Job Title": latest_child["filters"]["Job Title"],
            "Keyword": combined_keyword_filter
        },
        "children": []
    }

    latest_child["children"].append(new_child)

    return root

In [17]:
# Global variable to store the latest IDs
latest_ids = []

keywords = 3
root = create_level_zero(matrix)

for i in range(1, keywords + 1):
    root = create_keyword_levels(root, matrix, i)
    
    latest_child = root["children"][-1]
    while latest_child["children"]:
        latest_child = latest_child["children"][-1]  # Traverse to the most recent child

    filters = latest_child["filters"]
    ids, profiles_count = run_api_search(filters, bearer)

    # Store only the latest iteration's IDs globally
    latest_ids = ids  

    print(f"------------------------------------------------------------------\nKeyword {i}:\n", latest_child)
    print(f"{profiles_count} profiles found.")

# Access latest_ids globally after the loop
print(f"\nLatest iteration IDs: {latest_ids}")


------------------------------------------------------------------
Keyword 1:
 {'filters': {'Location': '(New York)', 'Last Update': '(2025-01-04 23:38:20)', 'Job Title': '((Product Manager) OR (Product Owner) OR (Technical Product Manager))', 'Keyword': '((Agile) OR (Scrum) OR (Kanban))'}, 'children': []}
2801 profiles found.
------------------------------------------------------------------
Keyword 2:
 {'filters': {'Location': '(New York)', 'Last Update': '(2025-01-04 23:38:20)', 'Job Title': '((Product Manager) OR (Product Owner) OR (Technical Product Manager))', 'Keyword': '((Agile) OR (Scrum) OR (Kanban)) AND ((Roadmap) OR (Backlog) OR (Feature Prioritization))'}, 'children': []}
954 profiles found.
------------------------------------------------------------------
Keyword 3:
 {'filters': {'Location': '(New York)', 'Last Update': '(2025-01-04 23:38:20)', 'Job Title': '((Product Manager) OR (Product Owner) OR (Technical Product Manager))', 'Keyword': '((Agile) OR (Scrum) OR (Kanban

----

    Colecting Profiles

In [ ]:
url = "https://api.coresignal.com/cdapi/v1/professional_network/employee/search/filter"

payload = {
    "last_updated_gte": (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d %H:%M:%S"),
    "experience_date_from": "2000",
    "experience_date_to": "2025",
    "active_experience": True,
    "location": 'Virginia',
    "experience_title": 'Salesforce Developer'
}

headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer '
}

all_ids = []
next_page_after = None

while True:
    # Modify URL for pagination if needed
    request_url = url if not next_page_after else f"{url}?after={next_page_after}"

    response = requests.post(request_url, headers=headers, data=json.dumps(payload))

    if response.status_code != 200:
        print(f"Error: API request failed with status {response.status_code}")
        print(f"Response Text: {response.text}")
        break
    
    try:
        response_json = response.json()
        all_ids.extend(response_json)

        next_page_after = response.headers.get("x-next-page-after")

        if next_page_after and next_page_after.isdigit():
            next_page_after = int(next_page_after)  # Convert to integer
        else:
            break  # Stop if no more pages

    except json.JSONDecodeError:
        print("Error: Invalid JSON response")
        break

print(f"Total IDs Retrieved: {len(all_ids)}")


Total IDs Retrieved: 103


In [63]:
all_ids

[79413750,
 79843881,
 101283895,
 140849851,
 182002025,
 209911842,
 244450295,
 257570822,
 274600002,
 343397944,
 353473454,
 382647206,
 386099881,
 396576770,
 397749932,
 415419729,
 416512504,
 419022535,
 419223943,
 420426331,
 422407852,
 422491395,
 428405350,
 429618022,
 430678672,
 432489407,
 437070109,
 441231022,
 441861614,
 459230890,
 462368470,
 466231111,
 475254513,
 484617684,
 488820491,
 489827171,
 490240515,
 495225563,
 496525588,
 500891287,
 503913830,
 504495794,
 507723336,
 512094722,
 515599178,
 517544662,
 521854226,
 527006299,
 563996517,
 570069628,
 570440778,
 571324732,
 572489560,
 572659693,
 573779070,
 574024905,
 574081411,
 574276728,
 576228270,
 581076135,
 581319377,
 604875941,
 612332970,
 612935438,
 616334015,
 618297747,
 628947006,
 634993378,
 635293823,
 635470879,
 635538792,
 635576249,
 635946039,
 637187356,
 638425316,
 639438627,
 640269638,
 645616263,
 649279963,
 683403120,
 684132560,
 696238688,
 736837056,
 74541

    Extracting Data from JSON profiles:

In [53]:
def collect_and_extract_profiles(all_ids):
    final_df = pd.DataFrame()  

    for i in all_ids:
        profile = collect_employees([i]) 
        if not profile:
            continue
        
        profile_df = extract_linkedin_profile(profile)  

        if not final_df.empty and profile_df["url"].iloc[0] in final_df["url"].values:
            print(f"Skipping duplicate profile: {profile_df['name'].iloc[0]}")
            continue

        final_df = pd.concat([final_df, profile_df], ignore_index=True)

    return final_df 

In [64]:
profiles_df = collect_and_extract_profiles(all_ids)

In [65]:
profiles_df.head()

,id,name,title,headline,url,shorthand_name,created,last_updated,logo_url,summary,location,country,connections,recommendations,industry,experience_count,experiences,education,awards,certifications,courses,groups,interests,languages,organizations,patents,projects,publications,skills,test_scores,volunteering_cares,volunteering_opportunities,volunteering_positions,volunteering_supports
0,79413750,Nithin Sinkaran,None,--,https://www.linkedin.com/in/nithinsinkaran,nithinsinkaran,2016-08-05 05:51:19,2025-01-07 05:45:53,https://media.licdn.com/dms/image/v2/C4E03AQEt...,None,"Falls Church, Virginia, United States",United States,209,[],Computer Software,4,"[{'id': 81315819282, 'member_id': 79413750, 't...","[{'id': 2117352658, 'member_id': 79413750, 'ti...",[],[],[],[],[],"[{'id': 192294729, 'member_id': 79413750, 'lan...",[],[],[],[],[],[],[],[],[],[]
1,79843881,Darryl Ingalls,Software Development Manager Extraordinaire,Software Development Manager Extraordinaire,https://www.linkedin.com/in/darryl-ingalls-749...,darryl-ingalls-749a4816,2016-08-05 06:10:00,2025-01-25 11:01:56,https://media.licdn.com/dms/image/v2/C4E03AQHt...,"Highly skilled, knowledgeable Senior Software ...","Richmond, Virginia, United States",United States,268,"[{'id': 141093812, 'member_id': 79843881, 'rec...",Computer Software,19,"[{'id': 1593139287345, 'member_id': 79843881, ...","[{'id': 2565245073, 'member_id': 79843881, 'ti...",[],"[{'id': 955365177, 'member_id': 79843881, 'nam...",[],[],[],[],[],[],[],[],[],[],[],[],[],[]
2,101283895,Jerry Pauly,None,Salesforce Developer at RTS Labs,https://www.linkedin.com/in/jerry-pauly-29a49811,jerry-pauly-29a49811,2016-08-06 06:23:37,2025-01-30 10:21:03,https://media.licdn.com/dms/image/v2/C4E03AQGF...,None,"Glen Allen, Virginia, United States",United States,410,[],Hospital & Health Care,8,"[{'id': 321831926820, 'member_id': 101283895, ...","[{'id': 2561663921, 'member_id': 101283895, 't...",[],"[{'id': 1058888781, 'member_id': 101283895, 'n...",[],[],[],[],[],[],[],[],[],[],[],[],[],[]
3,140849851,Prashanthi Nammi,None,None,https://www.linkedin.com/in/prashanthi-nammi-0...,prashanthi-nammi-099b3582,2016-08-08 17:11:22,2025-01-23 07:26:45,https://media.licdn.com/dms/image/v2/C5103AQHA...,None,"Herndon, Virginia, United States",United States,65535,[],Information Technology & Services,3,"[{'id': 98491047009, 'member_id': 140849851, '...","[{'id': 89119809, 'member_id': 140849851, 'tit...",[],"[{'id': 1072127227, 'member_id': 140849851, 'n...",[],[],[],[],[],[],[],[],[],[],[],[],[],[]
4,182002025,Deepak Thadani,Salesforce Developer at The CloudFountain Inc,Salesforce Developer at The CloudFountain Inc,https://www.linkedin.com/in/dthadani,dthadani,2016-08-13 03:10:04,2025-01-07 21:07:43,https://static.licdn.com/aero-v1/sc/h/9c8pery4...,None,"Lorton, Virginia, United States",United States,134,[],Medical Device,4,"[{'id': 605297160242, 'member_id': 182002025, ...","[{'id': 116693203, 'member_id': 182002025, 'ti...",[],[],[],[],[],"[{'id': 177331947, 'member_id': 182002025, 'la...",[],[],[],[],[],[],[],[],[],[]


In [66]:
profiles_df.shape

(103, 34)

In [67]:
store_profiles_in_db(profiles_df)

Profiles successfully stored in the database.


In [68]:
def display_entire_db(db_path="profiles.db"):
    conn = sqlite3.connect(db_path)
    df = pd.read_sql("SELECT * FROM profiles", conn)
    conn.close()

    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    pd.set_option('display.width', 1000)
    print(df)

display_entire_db()

            id                        name                                              title                                           headline                                                url                      shorthand_name              created         last_updated                                           logo_url                                            summary                                      location        country connections                                    recommendations                            industry experience_count                                        experiences                                          education                                             awards                                     certifications                                            courses groups interests                                          languages                                      organizations patents                                           projects  \
0    789852

----